In [25]:
import scanpy as sc
import anndata as ad
import pandas as pd

In [3]:
!curl -o "raw_data/expression_matrix.tsv.gz" "http://singlecell.charite.de/cellbrowser/pancreas/Neonatal_Pancreas/exprMatrix.tsv.gz"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.7M  100 10.7M    0     0  20.9M      0 --:--:-- --:--:-- --:--:-- 21.0M


In [7]:
!curl -o "raw_data/meta_data.tsv" "http://singlecell.charite.de/cellbrowser/pancreas/Neonatal_Pancreas/meta.tsv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  558k  100  558k    0     0  2175k      0 --:--:-- --:--:-- --:--:-- 2181k


In [9]:
! gzip -d raw_data/expression_matrix.tsv.gz

In [14]:
! tr '\t' ',' < raw_data/expression_matrix.tsv > raw_data/expression_matrix.csv

In [24]:
! rm raw_data/expression_matrix.tsv

In [15]:
adata = ad.read_csv("raw_data/expression_matrix.csv")

In [23]:
adata = adata.transpose()

In [26]:
obs = pd.read_table("raw_data/meta_data.tsv")

In [28]:
obs.index = obs.Cell.rename("UMI")

In [32]:
obs.drop("Cell", axis=1, inplace=True)

In [37]:
adata.obs = adata.obs.merge(obs, left_index=True, right_index=True)

In [43]:
adata.obs["dataset"] = "neonatal"
adata.obs["organism"] = "Homo sapiens"
adata.obs["disease"] = "healthy"
adata.obs.rename({"Cluster": "cell_type"}, axis = 1, inplace = True)

In [46]:
adata.obs = adata.obs[['disease', 'dataset', 'organism', "cell_type"]]

In [47]:
adata

AnnData object with n_obs × n_vars = 10528 × 25936
    obs: 'disease', 'dataset', 'organism', 'cell_type'

In [50]:
adata.write_h5ad("neonatal_human_war_counts.h5ad")

In [52]:
sc.pp.normalize_per_cell(adata)

In [53]:
sc.pp.log1p(adata)

In [54]:
adata.write_h5ad("neonatal_human_normal_per_cell_log_trans.h5ad")